In [1]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import numpy as np


In [9]:
pwd

'd:\\Python\\LLM_all\\RNN_basic'

In [7]:
# text = """The quick brown fox jumps over the lazy dog. The quick brown fox is very quick."""


with open('d:\\Python\\LLM_all\\RNN_basic\\ska_all.txt', 'r') as file:
    text = file.read()

In [9]:
text=text[0:100000]

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Convert text to sequences of integers
input_sequences = []
for line in text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences to ensure uniform input size
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Split the data into predictors and label
X, y = input_sequences[:,:-1], input_sequences[:,-1]

# Convert labels to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [11]:
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(SimpleRNN(150, return_sequences=False))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


c:\Users\Saurabh Parmar\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(X, y, epochs=10, verbose=1)
import pickle
with open('rnn_model.pkl', 'wb') as file:
    pickle.dump(model, file)

Epoch 1/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.0213 - loss: 6.9373
Epoch 2/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.0296 - loss: 6.5188
Epoch 3/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.0372 - loss: 6.3016
Epoch 4/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.0466 - loss: 6.0369
Epoch 5/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.0607 - loss: 5.7596
Epoch 6/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.0736 - loss: 5.4285
Epoch 7/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.0890 - loss: 5.1104
Epoch 8/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.1133 - loss: 4.7331
Epoch 9/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.1537 - loss: 4.4375
Epoch 10/10
529/529 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - accuracy: 0.2029 - loss: 4.0896


In [13]:
with open('rnn_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [14]:
def what_would_shakespeare_say(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = loaded_model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=1)[0]
        predicted_word = tokenizer.index_word[predicted_word_index]
        seed_text += " " + predicted_word
    return seed_text




In [15]:
seed_text = "The trouser"
next_words = 15
print(what_would_shakespeare_say(seed_text, next_words, max_sequence_len))

The trouser time is thy sweet will be up the watery main increasing store to be and


Num GPUs Available:  0
GPUs Available:  []
